<a href="https://colab.research.google.com/github/nashyvant/ai_projects/blob/main/project3_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
input_train_data = []
output_train_data = []

In [3]:
# importing "heapq" to implement heap queue
import heapq
import sys
import copy
import math
import random
import numpy as np
import torch

#from utility import weight
from datetime import datetime

#import shared_tensors

# Get the current date and time
now = datetime.now()

# Format the date and time as a string
current_time = now.strftime("%Y-%m-%d %H:%M:%S")

# Print the current date and time
print("Current Date and Time:", current_time)

# Define the directions for moving in the matrix
DIRECTIONS = [(-1, 0), (1, 0), (0, -1), (0, 1)]
# Label mapping for one-hot encoding
label_mapping = {"up": 0, "down": 1, "left": 2, "right": 3, "sense": 4}

d = int(40)

ship_map = [
    [0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1],
    [1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1],
    [1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1],
    [1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0],
    [1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1],
    [1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1],
    [1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1],
    [0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1],
    [0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1],
    [1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0],
    [1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1],
    [1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1],
    [1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1],
    [0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1],
    [0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
    [1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1],
    [1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0],
    [0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0],
    [1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0],
    [1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1],
    [0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0],
    [1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0],
    [1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0],
    [1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0],
    [0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1],
    [1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0],
    [0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1],
    [1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0],
    [1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1],
    [1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0],
    [1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0],
    [0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0],
    [1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1],
    [1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
    [1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1],
    [0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1]
]

####################################################################
######## creates DxD matrix with probabilitic knowledge base of ####
######## where the mouse is ########################################
####################################################################
def create_pkb(ship_map):
    row, col = (len(ship_map), len(ship_map[0]))
    num_open_cells = 0
    for i in range(0, len(ship_map)):
        for j in range(0, len(ship_map[i])):
            if(ship_map[i][j] == 1):
                num_open_cells += 1
    uniform_dist = 1/(num_open_cells)
    pkb = [[uniform_dist for i in range(col)] for j in range(row)]

    for i in range(0, len(pkb)):
        for j in range(0, len(pkb[i])):
            if(ship_map[i][j] == 0):
                pkb[i][j] = 0

    return pkb

def create_bot_fire_button(ship_map):
    rand_row = random.randint(0, len(ship_map)-1)
    rand_col = random.randint(0, len(ship_map[0])-1)

    while(ship_map[rand_row][rand_col] != 1):
        rand_row = random.randint(0, len(ship_map)-1)
        rand_col = random.randint(0, len(ship_map[0])-1)
    bot_coord = (rand_row, rand_col)

    while((ship_map[rand_row][rand_col] == 0) or (bot_coord == (rand_row, rand_col))):
        rand_row = random.randint(0, len(ship_map)-1)
        rand_col = random.randint(0, len(ship_map[0])-1)
    button_coord = (rand_row, rand_col)

    while(ship_map[rand_row][rand_col] == 0
          or ((rand_row, rand_col) == bot_coord
          or (rand_row, rand_col) == button_coord)):
        rand_row = random.randint(0, len(ship_map)-1)
        rand_col = random.randint(0, len(ship_map[0])-1)
    fire_coord = (rand_row, rand_col)
    return bot_coord, button_coord, fire_coord

def sample_bit(p):
    if random.random() <= p :
        return 1
    else:
        return 0

def get_path(parent, curr):
    complete_path = [curr]
    steps = 0
    while curr in parent:
        curr = parent[curr]
        complete_path.append(curr)
        steps += 1
    complete_path.reverse()
    return complete_path, steps

pkb = create_pkb(ship_map)

rows = len(ship_map)
cols = len(ship_map[0])

def print_bot_map(bot_coord):
    #print("bot is now at:", bot_coord)
    bot_map = [[0 for i in range(d)] for j in range(d)]
    bot_map[bot_coord[0]][bot_coord[1]] = 1
    return bot_map

def heuristic(a, b):
    return round((abs(a[0] - b[0]) + abs(a[1] - b[1])), 8)

def update_known_pkb(bot_coord): # we are at the current cell and know if the mouse is not here, update PKB
    pkb[bot_coord[0]][bot_coord[1]] = 0
    return

def update_pkb(bot_coord, is_beep, alpha):
    sum = 0
    for i in range(0, len(pkb)):
        for j in range(0, len(pkb[i])):
            d = heuristic(bot_coord, (i, j))
            updated_prob = 0
            if(is_beep):
                updated_prob = round(pkb[i][j]*(math.exp(-alpha*(d-1))), 8)
            else:
                updated_prob = round(pkb[i][j]*(1-math.exp(-alpha*(d-1))), 8)
            if(updated_prob < 0 or updated_prob <= -0):
                pkb[i][j] = 0.0
            else:
                pkb[i][j] = updated_prob
            sum += pkb[i][j]

    for i in range(0, len(pkb)):
        for j in range(0, len(pkb[i])):
            pkb[i][j] = round(pkb[i][j]/sum, 8)

    return

def mouse_sensor(bot_coord, mouse_coord, alpha):
    d = heuristic(bot_coord, mouse_coord)
    beep_prob = math.exp(-alpha*(d-1))
    return sample_bit(beep_prob)

def sense(bot_coord, mouse_coord, alpha):
    is_beep = mouse_sensor(bot_coord, mouse_coord, alpha)
    update_pkb(bot_coord, is_beep, alpha)
    return is_beep

def get_max_likelihood_neighbor(bot_coord):
    new_bot_coord = []
    max_prob = 0
    for x, y in DIRECTIONS: #first get the max P
        neighbor = (bot_coord[0]+x, bot_coord[1]+y)
        if (0 <= neighbor[0] < d and 0 <= neighbor[1] < d) and (ship_map[neighbor[0]][neighbor[1]] == 1): #ignores fire and block cells
            if(pkb[neighbor[0]][neighbor[1]] > max_prob):
                max_prob = pkb[neighbor[0]][neighbor[1]] #gets the max P across 4 neighbors

    for x, y in DIRECTIONS: #now get all the neighbors with the same max probability
        neighbor = (bot_coord[0]+x, bot_coord[1]+y)
        if (0 <= neighbor[0] < d and 0 <= neighbor[1] < d) and (ship_map[neighbor[0]][neighbor[1]] == 1): #ignores fire and block cells
            if(pkb[neighbor[0]][neighbor[1]]  == max_prob):
                new_bot_coord.append(neighbor)

    #print("number of max likelihood neighbors: ", new_bot_coord)
    r = random.randint(0, len(new_bot_coord)-1)
    #print("picking neighbor:", new_bot_coord[r])
    return new_bot_coord[r]

def get_overall_max_likelihood_coord(bot_coord):
    new_bot_coord = []
    max_prob = pkb[bot_coord[0]][bot_coord[1]]
    for i in range(0, len(pkb)):
        for j in range(0, len(pkb[i])):
            if(pkb[i][j] > max_prob):
                max_prob = pkb[i][j] #gets the max P

    for i in range(0, len(pkb)):
        for j in range(0, len(pkb[i])):
            if(pkb[i][j] == max_prob):
                new_bot_coord.append((i, j))

    #print("number of overall max likelihood cells: ", new_bot_coord)
    r = random.randint(0, len(new_bot_coord)-1)
    #print("picking max P cell:", new_bot_coord[r])
    return new_bot_coord[r]

def a_star(ship_map, start, goal):
    myheap = []
    heapq.heappush(myheap, (0, start))
    parent = {}
    cost = {start : 0}
    est_total_cost = {start : heuristic(start, goal)}

    while myheap:
        curr_cell = heapq.heappop(myheap)
        if(curr_cell[1] == goal):
            return get_path(parent, curr_cell[1])

        i = curr_cell[1][0]
        j = curr_cell[1][1]


        for x, y in DIRECTIONS:
            neighbor = (curr_cell[1][0]+x, curr_cell[1][1]+y)
            if (0 <= neighbor[0] < d and 0 <= neighbor[1] < d) and (ship_map[neighbor[0]][neighbor[1]] == 1): #block and zero prob cells
                est_curr_cost = cost[curr_cell[1]]+1

                if neighbor not in cost or est_curr_cost < cost[neighbor]:
                    parent[neighbor] = curr_cell[1]
                    cost[neighbor] = est_curr_cost
                    est_total_cost[neighbor] = est_curr_cost+heuristic(neighbor, goal)
                    heapq.heappush(myheap, (est_total_cost[neighbor], neighbor))

    return None, 0

def run_bot(ship_map, bot_coord, mouse_coord, alpha, input_data, output_data, bot_num):
    total_actions = 0
    while(True):
        bot_map = print_bot_map(bot_coord)
        # Stack the matrices into a 3x40x40 tensor
        input_tensor = np.stack((ship_map, pkb, bot_map), axis = 0)
        # Store the input tensor
        input_data.append(torch.tensor(input_tensor, dtype=torch.float32))
        if(bot_coord == mouse_coord):
            #print("bot found the mouse!")
            output = "sense"
            output_index = label_mapping[output]
            output_data.append(output_index)
            return 1, total_actions

        update_known_pkb(bot_coord) # we are at the current cell and know if the mouse is not here, update PKB

        output = "sense"
        output_index = label_mapping[output]
        output_data.append(output_index)

        is_beep = sense(bot_coord, mouse_coord, alpha)
        #print("At ", bot_coord, " bot beeped:", is_beep)
        total_actions = total_actions+1 # for sensing
        next_bot_coord = get_overall_max_likelihood_coord(bot_coord)
        if(next_bot_coord == bot_coord): # need new coordinates so looking for neighbors
            #print("new coords same as old bot cords", next_bot_coord, bot_coord)
            next_bot_coord = get_max_likelihood_neighbor(bot_coord)

        path, steps = a_star(ship_map, bot_coord, next_bot_coord)
        if path is None:
            #print("ALERT! Cannot find a_star path from ", bot_coord, " to ", next_bot_coord)
            return 0, 0
        #else:
            #print(f"Path to next highest probability: {path}")

        k = 1
        while(k < len(path) and pkb[path[k][0]][path[k][1]] <= 0):
            #print("Skip sensing bot_coord: ", path[k], " its PKB is:", pkb[path[k][0]][path[k][1]])

            bot_map = print_bot_map(bot_coord)

            # Stack the matrices into a 3x40x40 tensor
            input_tensor = np.stack((ship_map, pkb, bot_map), axis = 0)
            # Store the input tensor
            input_data.append(torch.tensor(input_tensor,  dtype=torch.float32))

            for x, y in DIRECTIONS: #first get the max P
                bot_move = (bot_coord[0]+x, bot_coord[1]+y)
                if (0 <= bot_move[0] < d and 0 <= bot_move[1] < d): #ignores fire and block cells
                    if(path[k] == bot_move):
                        if(x == -1 and y == 0):
                            output = "up"
                        if(x == +1 and y == 0):
                            output = "down"
                        if(x == 0 and y == -1):
                            output = "left"
                        if(x == 0 and y == +1):
                            output = "right"
                        break

            output_index = label_mapping[output]
            output_data.append(output_index)

            bot_coord = path[k]
            k  = k+1

        bot_map = print_bot_map(bot_coord)
        # Stack the matrices into a 3x40x40 tensor
        input_tensor = np.stack((ship_map, pkb, bot_map), axis = 0)
        # Store the input tensor
        input_data.append(torch.tensor(input_tensor, dtype=torch.float32))

        for x, y in DIRECTIONS: #first get the max P
            bot_move = (path[k-1][0]+x, path[k-1][1]+y)
            if (0 <= bot_move[0] < d and 0 <= bot_move[1] < d): #ignores fire and block cells
                if(path[k] == bot_move):
                  if(x == -1 and y == 0):
                    output = "up"
                  if(x == +1 and y == 0):
                    output = "down"
                  if(x == 0 and y == -1):
                    output = "left"
                  if(x == 0 and y == +1):
                     output = "right"
                  break
        output_index = label_mapping[output]
        output_data.append(output_index)

        bot_coord = path[k]
        total_actions = total_actions+1+(k-1) # move one step

def float_range(start, end, step):
    while start < end:
        yield start
        start += step

for alpha in float_range(0.1, 1, 0.1):
    for i in range(0, 30):
        pkb = create_pkb(ship_map)
        bot_coord = (9, 34)
        mouse_coord = (16, 17)
        #bot_coord, mouse_coord, __notneeded = create_bot_fire_button(ship_map)
        mission_success_2, total_actions_2 = run_bot(ship_map, bot_coord, mouse_coord, alpha, input_train_data, output_train_data,  2)
        #print("alpha is: ", alpha, " D is:", d)
        #print("bot @ ", bot_coord, " mouse @ ", mouse_coord)
        print("bot2 mission:", mission_success_2, "total # of actions", total_actions_2)

# Convert lists to tensors when done collecting data
input_train_data = torch.stack(input_train_data)
output_train_data = torch.tensor(output_train_data, dtype=torch.long)  # Make sure this is of type long

Current Date and Time: 2024-07-22 11:35:59
bot2 mission: 1 total # of actions 142
bot2 mission: 1 total # of actions 99
bot2 mission: 1 total # of actions 72
bot2 mission: 1 total # of actions 57
bot2 mission: 1 total # of actions 360
bot2 mission: 1 total # of actions 128
bot2 mission: 1 total # of actions 164
bot2 mission: 1 total # of actions 112
bot2 mission: 1 total # of actions 442
bot2 mission: 1 total # of actions 82
bot2 mission: 1 total # of actions 222
bot2 mission: 1 total # of actions 224
bot2 mission: 1 total # of actions 98
bot2 mission: 1 total # of actions 103
bot2 mission: 1 total # of actions 256
bot2 mission: 1 total # of actions 114
bot2 mission: 1 total # of actions 433
bot2 mission: 1 total # of actions 68
bot2 mission: 1 total # of actions 92
bot2 mission: 1 total # of actions 214
bot2 mission: 1 total # of actions 231
bot2 mission: 1 total # of actions 95
bot2 mission: 1 total # of actions 45
bot2 mission: 1 total # of actions 68
bot2 mission: 1 total # of acti

In [4]:
input_test_data = []
output_test_data = []

for alpha in float_range(0.1, 1, 0.1):
    for i in range(0, 15):
        pkb = create_pkb(ship_map)
        #bot_coord, mouse_coord, __notneeded = create_bot_fire_button(ship_map)
        bot_coord = (9, 34)
        mouse_coord = (16, 17)
        mission_success_2, total_actions_2 = run_bot(ship_map, bot_coord, mouse_coord, alpha, input_test_data, output_test_data,  2)
        #print("alpha is: ", alpha, " D is:", d)
        #print("bot @ ", bot_coord, " mouse @ ", mouse_coord)
        #print("bot2 mission:", mission_success_2, "total # of actions", total_actions_2)

# Convert lists to tensors when done collecting data
input_test_data = torch.stack(input_test_data)
output_test_data = torch.tensor(output_test_data, dtype=torch.long)  # Make sure this is of type long

bot2 mission: 1 total # of actions 169
bot2 mission: 1 total # of actions 168
bot2 mission: 1 total # of actions 220
bot2 mission: 1 total # of actions 200
bot2 mission: 1 total # of actions 80
bot2 mission: 1 total # of actions 123
bot2 mission: 1 total # of actions 375
bot2 mission: 1 total # of actions 62
bot2 mission: 1 total # of actions 95
bot2 mission: 1 total # of actions 156
bot2 mission: 1 total # of actions 305
bot2 mission: 1 total # of actions 78
bot2 mission: 1 total # of actions 148
bot2 mission: 1 total # of actions 246
bot2 mission: 1 total # of actions 83
bot2 mission: 1 total # of actions 123
bot2 mission: 1 total # of actions 68
bot2 mission: 1 total # of actions 142
bot2 mission: 1 total # of actions 93
bot2 mission: 1 total # of actions 131
bot2 mission: 1 total # of actions 78
bot2 mission: 1 total # of actions 80
bot2 mission: 1 total # of actions 220
bot2 mission: 1 total # of actions 78
bot2 mission: 1 total # of actions 98
bot2 mission: 1 total # of actions 5

In [5]:
# Print shapes to verify
print("Input data shape:", input_train_data.shape)  # Should be (#actions+1, 3, 40, 40)
print("Output data shape:", output_train_data.shape)  # Should be (#actions+1, 5)

print("Input test data shape:", input_test_data.shape)  # Should be (#actions+1, 3, 40, 40)
print("Output test data shape:", output_test_data.shape)  # Should be (#actions+1, 5)

Input data shape: torch.Size([67872, 3, 40, 40])
Output data shape: torch.Size([67872])
Input test data shape: torch.Size([36407, 3, 40, 40])
Output test data shape: torch.Size([36407])


In [6]:
import torch.nn as nn
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.optim as optim
import random

class BotMoves(nn.Module):
  def __init__(self):
    super(BotMoves, self).__init__()

    self.conv_1 = torch.nn.Conv2d( in_channels = 3, out_channels = 30, kernel_size = (3,3), stride = 1, bias = True )
    # This will compute 3x3 local features, and compute 5 different features at every possible 3x3 location (at stride 1)
    #(40-3)/1 +1 (incoming_matrix - window size )/stride + 1
    #30x38x38

    self.conv_2 = torch.nn.Conv2d( in_channels = 30, out_channels = 20, kernel_size = (2,2), stride = 2, bias = True )
    # This will compute 20 different features on 2x2 windows at every possible location (at stride 2)
    #38-2/2 +1 = 19
    #20x19x19

    self.dropout = torch.nn.Dropout(0.2) #0.5 to 0.2

    self.linear_1 = torch.nn.Linear( in_features = 20*19*19, out_features = 10, bias = True )
    # We can flatten it down to a vector of size 5*19*19, and pass it into a linear layer to get the logits we need for classification

    self.linear_2 = torch.nn.Linear( in_features = 10, out_features = 5, bias = True )

  def forward(self, input_tensor):
    reshaped = torch.reshape( input_tensor, (-1, 3, 40, 40) )

    conv_results = self.conv_1( reshaped )
    conv_results = torch.nn.ReLU()( conv_results )

    conv_results = self.conv_2( conv_results )
    conv_results = torch.nn.ReLU()( conv_results )

    flattened = torch.nn.Flatten()( conv_results )

    drop_out = self.dropout( flattened )

    logits_1 = self.linear_1( drop_out )

    logits = torch.nn.ReLU()( logits_1 )

    logits = self.linear_2(logits)

    probabilities = nn.Softmax( dim = 1 )( logits )
    return logits, probabilities

In [7]:
model = BotMoves()

optimizer = optim.Adam( model.parameters(), lr = 0.005 ) #0.005 gave me the best loss values
#optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [8]:
loss_function = nn.CrossEntropyLoss()

In [9]:
def confusion_matrix(model, x, y):
  batch_size = 64
  model.eval() # Depending on the model, it may have 'training specific' properties that we should turn off when we are trying to evaluate the model.

  identification_counts = np.zeros( shape = (5,5), dtype = np.int32 )

  n = x.shape[0]

  # Process the dataset in batches
  total_correct = 0
  for start_idx in range(0, n, batch_size):
        end_idx = min(start_idx + batch_size, n)
        x_batch = x[start_idx:end_idx]
        y_batch = y[start_idx:end_idx]

        with torch.no_grad():  # Disable gradient calculation for evaluation
            logits, probabilities = model(x_batch)
            predicted_classes = torch.argmax(probabilities, dim=1)

        for i in range(x_batch.shape[0]):
            actual_class = y_batch[i].item()
            predicted_class = predicted_classes[i].item()
            identification_counts[actual_class, predicted_class] += 1

  for i in range(5):
      total_correct += identification_counts[i, i] # Summing across the diagonal, counting up the correct identifications

  accuracy = total_correct / np.sum( identification_counts )
  print("total correct:", total_correct, " total:", identification_counts)
  return identification_counts, accuracy # Return the confusion matrix and the accuracy

In [15]:
from torch.utils.data import DataLoader, TensorDataset

batch_size = 64 #1024, loss started high at 1.60 and went down to 1.59 in the next step
dataset = TensorDataset(input_train_data, output_train_data)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True) # avoided the random increase in loss at various loss points

model.train()
for epoch in range(10):
    total_loss = 0
    for x_batch, y_batch in data_loader:
        optimizer.zero_grad()

        logits, probabilities = model(x_batch)

        loss = loss_function(logits, y_batch)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}, Average Loss per Data Point: {total_loss / len(data_loader)}")

Epoch 1, Average Loss per Data Point: 0.7084644131229016
Epoch 2, Average Loss per Data Point: 0.712643989490856
Epoch 3, Average Loss per Data Point: 0.7108220674877005
Epoch 4, Average Loss per Data Point: 0.7115927204896997
Epoch 5, Average Loss per Data Point: 0.7125622473383264
Epoch 6, Average Loss per Data Point: 0.7050758373557566
Epoch 7, Average Loss per Data Point: 0.698732123326626
Epoch 8, Average Loss per Data Point: 0.7052331798544029
Epoch 9, Average Loss per Data Point: 0.6994516713567549
Epoch 10, Average Loss per Data Point: 0.723322322102339


In [16]:
confusion_matrix(model, input_train_data, output_train_data)

total correct: 43838  total: [[ 5281  4033  1831   929     9]
 [ 1281 10246  2348   280    28]
 [  301  1101 15721   521    25]
 [ 1098  1813  6128  3518    12]
 [  109   768  1247   172  9072]]


(array([[ 5281,  4033,  1831,   929,     9],
        [ 1281, 10246,  2348,   280,    28],
        [  301,  1101, 15721,   521,    25],
        [ 1098,  1813,  6128,  3518,    12],
        [  109,   768,  1247,   172,  9072]], dtype=int32),
 0.6458922677982084)

In [17]:
confusion_matrix(model, input_test_data, output_test_data)

total correct: 22330  total: [[2599 2285 1055  553   20]
 [ 793 5239 1291  204   35]
 [ 205  640 8306  339   32]
 [ 708 1020 3520 1691   33]
 [  82  442  700  120 4495]]


(array([[2599, 2285, 1055,  553,   20],
        [ 793, 5239, 1291,  204,   35],
        [ 205,  640, 8306,  339,   32],
        [ 708, 1020, 3520, 1691,   33],
        [  82,  442,  700,  120, 4495]], dtype=int32),
 0.6133435877715824)